In [25]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import os

In [26]:
path_data =r'C:\Users\patri\eSIM2020\data'

In [27]:
list_data = []

for file in os.listdir(path_data):
    if file.endswith(".csv"):
        list_data.append(os.path.join(path_data, file))
        
list_data

['C:\\Users\\patri\\eSIM2020\\data\\2016-2017_chilledwater.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\2016-2017_electricity.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\2016-2017_gas.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\2016-2017_hotwater.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\2016-2017_irrigation.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\2016-2017_solar.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\2016-2017_steam.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\2016-2017_water.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\2016-2017_weather.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\building_metadata.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\metadata.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\sample_submission.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\site_metadata.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\test.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\train.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\weather_test.csv',
 'C:\\Users\\patri\\eSIM2020\\data\\weather_train.csv']

In [28]:
list_data_powerMeter = [os.path.join(path_data, '2016-2017_chilledwater.csv'),
                        os.path.join(path_data, '2016-2017_electricity.csv'),
                        os.path.join(path_data, '2016-2017_hotwater.csv'),
                        os.path.join(path_data, '2016-2017_steam.csv')]

In [29]:
df_meter = pd.DataFrame()

for path_powerMeter in list_data_powerMeter:
    powerMeter_type = path_powerMeter.split('\\')[5].split('_')[1].split('.')[0]
    df_temp = pd.read_csv(path_powerMeter,index_col='timestamp',parse_dates=True)

    df_temp = df_temp[df_temp.loc['2016'].columns[(~df_temp.loc['2016'].isna()).sum()/len(df_temp)>0.0]]
    df_temp = df_temp[df_temp.loc['2017'].columns[(~df_temp.loc['2017'].isna()).sum()/len(df_temp)>0.0]]

    #Preprocessing: remove missing data
    df_temp['Date'] = pd.to_datetime(df_temp.index.date)
    for name_meter in df_temp.drop('Date',axis=1).columns:
        try:
            df_daily = df_temp[name_meter].copy()
            df_daily = df_daily.resample('D').std()
            df_daily = df_daily.loc['2016']
            list_abnormalDate = list(df_daily[df_daily==0].index.strftime('%Y-%m-%d'))      
            df_temp.loc[df_temp['Date'].isin(list_abnormalDate), name_meter] = np.nan
        except:
            print('error')
    df_temp = df_temp.drop('Date', axis=1)  
        
    df_temp = df_temp.reset_index().melt(id_vars='timestamp').rename(columns={'variable':'building_id','value':'meter_reading'})
    df_temp['meter'] = powerMeter_type
    #df_temp = df_temp.dropna(subset=['meter_reading'])
    df_meter = pd.concat([df_meter, df_temp],axis=0,ignore_index=True)
    
df_meter

,timestamp,building_id,meter_reading,meter
0,2016-01-01 00:00:00,Gwyneth_Panther_Office,NaN,chilledwater
1,2016-01-01 01:00:00,Gwyneth_Panther_Office,NaN,chilledwater
2,2016-01-01 02:00:00,Gwyneth_Panther_Office,NaN,chilledwater
3,2016-01-01 03:00:00,Gwyneth_Panther_Office,NaN,chilledwater
4,2016-01-01 04:00:00,Gwyneth_Panther_Office,NaN,chilledwater
...,...,...,...,...
44579299,2017-12-31 19:00:00,Peter_Cockatoo_Publicservices,877.3558,steam
44579300,2017-12-31 20:00:00,Peter_Cockatoo_Publicservices,879.8632,steam
44579301,2017-12-31 21:00:00,Peter_Cockatoo_Publicservices,801.8268,steam
44579302,2017-12-31 22:00:00,Peter_Cockatoo_Publicservices,802.4247,steam


In [30]:
'''
df_meter = pd.DataFrame()

for path_powerMeter in list_data_powerMeter:
    powerMeter_type = path_powerMeter.split('\\')[5].split('_')[1].split('.')[0]
    df_temp = pd.read_csv(path_powerMeter,index_col='timestamp',parse_dates=True)
    
    #Remove meters with few data (<80%)    
    df_temp = df_temp[df_temp.columns[(~df_temp.isna()).sum()/len(df_temp)>0.8]]
    
    #Preprocessing: remove missing data
    df_temp['Date'] = pd.to_datetime(df_temp.index.date)
    for name_meter in df_temp.drop('Date',axis=1).columns:
        try:
            df_daily = df_temp[name_meter].copy()
            df_daily = df_daily.resample('D').std()
            df_daily = df_daily.loc['2016']
            list_abnormalDate = list(df_daily[df_daily==0].index.strftime('%Y-%m-%d'))      
            df_temp.loc[df_temp['Date'].isin(list_abnormalDate), name_meter] = np.nan
        except:
            print('error')
    df_temp = df_temp.drop('Date', axis=1)  
    
    df_temp = df_temp.reset_index().melt(id_vars='timestamp').rename(columns={'variable':'building_id','value':'meter_reading'})
    df_temp['meter'] = powerMeter_type
    #df_temp = df_temp.dropna(subset=['meter_reading'])
    df_meter = pd.concat([df_meter, df_temp],axis=0,ignore_index=True)
    
df_meter
'''

"\ndf_meter = pd.DataFrame()\n\nfor path_powerMeter in list_data_powerMeter:\n    powerMeter_type = path_powerMeter.split('\\')[5].split('_')[1].split('.')[0]\n    df_temp = pd.read_csv(path_powerMeter,index_col='timestamp',parse_dates=True)\n    \n    #Remove meters with few data (<80%)    \n    df_temp = df_temp[df_temp.columns[(~df_temp.isna()).sum()/len(df_temp)>0.8]]\n    \n    #Preprocessing: remove missing data\n    df_temp['Date'] = pd.to_datetime(df_temp.index.date)\n    for name_meter in df_temp.drop('Date',axis=1).columns:\n        try:\n            df_daily = df_temp[name_meter].copy()\n            df_daily = df_daily.resample('D').std()\n            df_daily = df_daily.loc['2016']\n            list_abnormalDate = list(df_daily[df_daily==0].index.strftime('%Y-%m-%d'))      \n            df_temp.loc[df_temp['Date'].isin(list_abnormalDate), name_meter] = np.nan\n        except:\n            print('error')\n    df_temp = df_temp.drop('Date', axis=1)  \n    \n    df_temp = df_t

In [31]:
df_meter.loc[((df_meter['building_id'].str.contains('Panther'))&(df_meter['timestamp']<='2016-05-20')), 'meter_reading'] = np.nan
df_meter = df_meter.reset_index().rename(columns={'index':'row_id'})
df_meter

,row_id,timestamp,building_id,meter_reading,meter
0,0,2016-01-01 00:00:00,Gwyneth_Panther_Office,NaN,chilledwater
1,1,2016-01-01 01:00:00,Gwyneth_Panther_Office,NaN,chilledwater
2,2,2016-01-01 02:00:00,Gwyneth_Panther_Office,NaN,chilledwater
3,3,2016-01-01 03:00:00,Gwyneth_Panther_Office,NaN,chilledwater
4,4,2016-01-01 04:00:00,Gwyneth_Panther_Office,NaN,chilledwater
...,...,...,...,...,...
44579299,44579299,2017-12-31 19:00:00,Peter_Cockatoo_Publicservices,877.3558,steam
44579300,44579300,2017-12-31 20:00:00,Peter_Cockatoo_Publicservices,879.8632,steam
44579301,44579301,2017-12-31 21:00:00,Peter_Cockatoo_Publicservices,801.8268,steam
44579302,44579302,2017-12-31 22:00:00,Peter_Cockatoo_Publicservices,802.4247,steam


In [32]:
building_meta = pd.read_csv(os.path.join(path_data, 'metadata.csv'))
building_meta = building_meta.merge(df_meter[['building_id','meter']].drop_duplicates(), on='building_id')
building_meta = building_meta.drop(['electricity', 'hotwater', 'chilledwater', 'steam', 'water',
                                    'irrigation', 'solar', 'gas'],axis=1)
building_meta

,building_id,site_id,primaryspaceusage,sqft,lat,lng,yearbuilt,numberoffloors,meter
0,Rosalie_Panther_Education,Panther,Education,7432.0,28.517689,-81.379039,2008.0,NaN,electricity
1,Misty_Panther_Education,Panther,Education,2720.0,28.517689,-81.379039,2004.0,NaN,electricity
2,Mattie_Panther_Education,Panther,Education,5376.0,28.517689,-81.379039,1991.0,NaN,electricity
3,Diann_Panther_Education,Panther,Education,23685.0,28.517689,-81.379039,2002.0,NaN,electricity
4,Gina_Panther_Education,Panther,Education,116607.0,28.517689,-81.379039,1975.0,NaN,electricity
...,...,...,...,...,...,...,...,...,...
2536,Lucie_Mouse_Healthcare,Mouse,Healthcare,185064.0,51.521939,-0.120069,1936.0,10.0,electricity
2537,Maude_Mouse_Healthcare,Mouse,Healthcare,58114.0,51.521939,-0.120069,1954.0,6.0,electricity
2538,Beau_Mouse_Healthcare,Mouse,Healthcare,18826.0,51.521939,-0.120069,NaN,8.0,electricity
2539,Casey_Mouse_Lodging/residential,Mouse,Lodging/residential,37555.0,51.521939,-0.120069,2006.0,7.0,electricity


In [33]:
df_weather = pd.read_csv(os.path.join(path_data, '2016-2017_weather.csv'))
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'])
df_weather.set_index('timestamp', inplace=True)

df_weather

,site_id,apparentTemperature,cloudCover,dewPoint,humidity,precipIntensity,precipType,pressure,summary,temperature,uvIndex,visibility,windBearing,windGust,windSpeed
timestamp,,,,,,,,,,,,,,,
2016-01-01 00:00:00,Panther,71.23,0.75,68.89,0.96,0.0000,NaN,1018.42,Mostly Cloudy,70.00,0.0,4.00,180.0,0.15,0.03
2016-01-01 01:00:00,Panther,70.86,0.75,68.61,0.96,0.0000,NaN,1018.91,Mostly Cloudy,69.66,0.0,3.00,180.0,0.16,0.01
2016-01-01 02:00:00,Panther,69.76,0.44,67.76,0.97,0.0000,NaN,1019.22,Partly Cloudy,68.63,0.0,2.58,197.0,0.03,0.03
2016-01-01 03:00:00,Panther,70.02,0.64,67.99,0.97,0.0000,NaN,1019.61,Mostly Cloudy,68.88,0.0,2.00,181.0,0.25,0.10
2016-01-01 04:00:00,Panther,72.83,0.54,70.49,0.97,0.0000,NaN,1020.09,Partly Cloudy,71.43,1.0,3.37,210.0,1.45,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 19:00:00,Mouse,43.49,0.21,42.14,0.84,0.0000,NaN,992.58,Clear,46.82,0.0,6.22,217.0,12.75,6.75
2017-12-31 20:00:00,Mouse,42.39,0.64,40.99,0.81,0.0000,NaN,992.34,Mostly Cloudy,46.44,0.0,6.22,215.0,15.05,8.11
2017-12-31 21:00:00,Mouse,41.91,0.63,40.87,0.80,0.0005,rain,992.28,Mostly Cloudy,46.73,0.0,6.22,233.0,19.59,10.20


In [34]:
'''
# # Count zero streak
train_df_black = df_meter[df_meter['timestamp']<'2017-01-01'].copy()
train_df_black = train_df_black.merge(building_meta, on=['building_id', 'meter'], how='left')
train_df_black = train_df_black.merge(df_weather, on=['site_id', 'timestamp'], how='left')

train_df_black['black_count']=0

for bid in train_df_black.building_id.unique():
    df = train_df_black[train_df_black.building_id==bid]
    for meter in df.meter.unique():
        dfm = df[df.meter == meter]
        b = (dfm.meter_reading == 0).astype(int)
        train_df_black.loc[(train_df_black.building_id==bid) & (train_df_black.meter == meter), 'black_count'] = b.groupby((~b.astype(bool)).cumsum()).cumsum()
        
df_meter = df_meter[df_meter['black_count'].fillna(0) < 24*black_day].drop('black_count', axis=1)
'''

"\n# # Count zero streak\ntrain_df_black = df_meter[df_meter['timestamp']<'2017-01-01'].copy()\ntrain_df_black = train_df_black.merge(building_meta, on=['building_id', 'meter'], how='left')\ntrain_df_black = train_df_black.merge(df_weather, on=['site_id', 'timestamp'], how='left')\n\ntrain_df_black['black_count']=0\n\nfor bid in train_df_black.building_id.unique():\n    df = train_df_black[train_df_black.building_id==bid]\n    for meter in df.meter.unique():\n        dfm = df[df.meter == meter]\n        b = (dfm.meter_reading == 0).astype(int)\n        train_df_black.loc[(train_df_black.building_id==bid) & (train_df_black.meter == meter), 'black_count'] = b.groupby((~b.astype(bool)).cumsum()).cumsum()\n        \ndf_meter = df_meter[df_meter['black_count'].fillna(0) < 24*black_day].drop('black_count', axis=1)\n"

In [35]:
df_meter.to_pickle(os.path.join(path_data, 'df_meter.pickle'))

In [36]:
building_meta.to_pickle(os.path.join(path_data, 'building_meta.pickle'))

In [37]:
df_weather.to_pickle(os.path.join(path_data, 'df_weather.pickle'))